# import stuff

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

this_directory = os.path.dirname(os.path.realpath('__file__')) # relative paths stopped working for some reason
data_directory = this_directory + '/data/'
processed_data_directory = data_directory + r'processed data' + '\\'
model_directory = data_directory + r'models' + '\\'

In [3]:
import sklearn
from sklearn.base import BaseEstimator, RegressorMixin, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer, HashingVectorizer
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.linear_model import Ridge, LinearRegression, SGDRegressor, LogisticRegression
from sklearn import datasets, tree, utils
from sklearn import model_selection, ensemble
from sklearn.decomposition import TruncatedSVD
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from sklearn.preprocessing import MinMaxScaler
from collections import Counter, OrderedDict
import math
from sklearn.naive_bayes import MultinomialNB, GaussianNB
import pickle as pkl

In [4]:
from yelpapi import YelpAPI
# get yelp api key stored in txt file
f = open(data_directory + 'yelp_api_key.txt','r')
api_key = f.read()
f.close()

In [5]:
import spacy
# nlp = spacy.load("en_core_web_sm")
from spacy.lang.en.stop_words import STOP_WORDS
STOP_WORDS = STOP_WORDS.union({'ll', 've'})
from sklearn.naive_bayes import MultinomialNB

# feature engineering

Trying a few new things beyond simple bag of words on individual reviews. Such a model is just not going to work because there are SO many reviews that don't mention cleanliness at all. I need to either weight reviews by how relevant they are to cleanliness, or better yet - don't train a model on irrelevant reviews. The key here is finding review "relevance" or a cleanliness sentiment for each review. This should be treated as an **unsupervised** problem, as the inspection scores only tell me about the overall cleanliness averaged over all reviews (within a certain time period).

In [6]:
df = pd.read_csv(processed_data_directory + 'reviews_nearest_score_no_limit_df.csv')
# df = pd.read_csv(processed_data_directory + 'reviews_nearest_score_2_years_df.csv')

In [13]:
df.groupby(['business_id','Score']).sum()

review_stars   stars
business_id            Score                      
-0TbQsmE2p1bhG30rgvK2Q 93.0            351   364.0
                       95.0              5     4.0
                       98.0              5     4.0
-4bP1UUgpZumIu6DZMaMzw 86.0           2233  2204.0
                       95.0             19    20.0
...                                    ...     ...
zo-POOSRX0wiVJlJyvujyA 100.0           175   164.0
zr40jIjFDIcZ3dzbSK53Ww 84.0             54    52.5
                       93.0            374   346.5
zubXuwlAEbvZd-MiLOgofQ 100.0           773   787.5
zvSUJ7J2-GcB-2E0W2ICbw 96.0             16    15.0

[6617 rows x 2 columns]

In [10]:
len(df.business_id.unique())

2506